In [ ]:
# ✅ Install required library
!pip install python-telegram-bot --upgrade

# ✅ Import required modules
import nest_asyncio
import asyncio
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (Application, CommandHandler, CallbackQueryHandler,
                          MessageHandler, filters, ConversationHandler, CallbackContext)
from datetime import datetime, timedelta

# ✅ Apply nest_asyncio for Google Colab
nest_asyncio.apply()

# ✅ Secure Bot Token and Admin Chat ID
BOT_TOKEN = "7715002025:AAE6Lm8bHp6DnIqsRTqAeng9BbmmCszyNg4"
ADMIN_CHAT_ID = 1349768533  # Your Admin Chat ID

# Define states
SELECT_DATE, SELECT_TIME, ENTER_NAME, ENTER_ERP, ENTER_ISSUE, ENTER_PHONE, CONFIRM = range(7)

# ✅ Start function with a Start button
async def start(update: Update, context: CallbackContext) -> int:
    # Create a "Start" button to begin the process
    keyboard = [[InlineKeyboardButton("Start", callback_data="start_button")]]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(
        "Welcome to the Psychological Counseling Bot! 🌿\nPlease click 'Start' to begin.",
        reply_markup=reply_markup
    )
    return ENTER_NAME  # Trigger the next state for the user's name

# ✅ Handle Start button click
async def start_button(update: Update, context: CallbackContext) -> int:
    await update.callback_query.answer()
    await update.callback_query.message.edit_text(
        "Please enter your full name to get started with the appointment booking."
    )
    return ENTER_NAME

# ✅ Get Student Name
async def enter_name(update: Update, context: CallbackContext) -> int:
    context.user_data['name'] = update.message.text
    await update.message.reply_text("Thank you! Now, please enter your ERP ID:")
    return ENTER_ERP

# ✅ Get ERP ID
async def enter_erp(update: Update, context: CallbackContext) -> int:
    context.user_data['erp'] = update.message.text
    return await select_date(update, context)

# ✅ Select Date
async def select_date(update: Update, context: CallbackContext) -> int:
    today = datetime.today()
    available_dates = [(today + timedelta(days=i)) for i in range(1, 31) if (today + timedelta(days=i)).weekday() in [4, 5]]

    keyboard = [[InlineKeyboardButton(date.strftime('%A, %d %B'), callback_data=date.strftime('%Y-%m-%d'))] for date in available_dates]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Please choose an available date:", reply_markup=reply_markup)
    return SELECT_DATE

# ✅ Date Selected
async def date_selected(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    await query.answer()
    context.user_data['date'] = query.data

    keyboard = [[InlineKeyboardButton(f"{hour}:30", callback_data=f"{hour}:30")] for hour in range(9, 16)]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await query.edit_message_text("Select a time slot:", reply_markup=reply_markup)
    return SELECT_TIME

# ✅ Time Selected
async def time_selected(update: Update, context: CallbackContext) -> int:
    query = update.callback_query
    await query.answer()
    context.user_data['time'] = query.data

    await query.edit_message_text("Please briefly describe the issue you'd like to discuss:")
    return ENTER_ISSUE

# ✅ Enter Issue
async def enter_issue(update: Update, context: CallbackContext) -> int:
    context.user_data['issue'] = update.message.text
    await update.message.reply_text("Please provide your phone number for contact:")
    return ENTER_PHONE

# ✅ Get Phone Number
async def enter_phone(update: Update, context: CallbackContext) -> int:
    context.user_data['phone'] = update.message.text
    details = (
        f"📝 **New Appointment Request:**\n\n"
        f"👤 Name: {context.user_data['name']}\n"
        f"🆔 ERP ID: {context.user_data['erp']}\n"
        f"📅 Date: {context.user_data['date']}\n"
        f"⏰ Time: {context.user_data['time']}\n"
        f"💬 Issue: {context.user_data['issue']}\n"
        f"📞 Contact: {context.user_data['phone']}"
    )

    # Check for appointment conflicts
    if check_appointment_conflict(context.user_data['date'], context.user_data['time']):
        await update.message.reply_text("❌ The selected time slot is already booked. Please choose another time.")
        return SELECT_TIME

    # Send details to Admin
    await context.bot.send_message(chat_id=ADMIN_CHAT_ID, text=details, parse_mode="Markdown")

    await update.message.reply_text(f"Here are your details:\n\n{details}\n\nConfirm? (Yes/No)")
    return CONFIRM

# ✅ Check for Appointment Conflicts (Dummy Function)
def check_appointment_conflict(date, time):
    # This is a dummy function. Replace it with actual logic to check conflicts.
    # You might store appointments in a database and check against that.
    return False  # Assuming no conflicts for now

# ✅ Confirmation
async def confirm(update: Update, context: CallbackContext) -> int:
    response = update.message.text.lower()
    if response == "yes":
        # Notify the student that the appointment is accepted
        await update.message.reply_text("✅ Your appointment has been successfully booked!")
    else:
        await update.message.reply_text("❌ Appointment cancelled. Restart with /start.")
    return ConversationHandler.END

# ✅ Cancel Function
async def cancel(update: Update, context: CallbackContext) -> int:
    await update.message.reply_text("❌ Conversation cancelled.")
    return ConversationHandler.END

# ✅ Main function to run the bot
async def main():
    application = Application.builder().token(BOT_TOKEN).build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start), CallbackQueryHandler(start_button, pattern="start_button")],
        states={
            ENTER_NAME: [MessageHandler(filters.TEXT & ~filters.COMMAND, enter_name)],
            ENTER_ERP: [MessageHandler(filters.TEXT & ~filters.COMMAND, enter_erp)],
            SELECT_DATE: [CallbackQueryHandler(date_selected, pattern=r"^\d{4}-\d{2}-\d{2}$")],
            SELECT_TIME: [CallbackQueryHandler(time_selected, pattern=r"^\d{1,2}:\d{2}$")],
            ENTER_ISSUE: [MessageHandler(filters.TEXT & ~filters.COMMAND, enter_issue)],
            ENTER_PHONE: [MessageHandler(filters.TEXT & ~filters.COMMAND, enter_phone)],
            CONFIRM: [MessageHandler(filters.TEXT & ~filters.COMMAND, confirm)],
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    application.add_handler(conv_handler)

    print("🤖 Bot is running...")
    await application.run_polling()

# ✅ Run the bot in Google Colab
await main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 10.7 MB/s eta 0:00:00


<ipython-input-1-1f3a1db92d7b>:138: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv_handler = ConversationHandler(


🤖 Bot is running...


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_req